- Which supplement type is more effective

- How dose levels affect tooth growth

- Whether there's an interaction between supplement type and dose

- The statistical significance of these effects

- The practical significance through effect sizes

## Main Effects Questions:
- 1. Does vitamin C supplementation significantly affect tooth growth?

> Is there a statistically significant overall effect?

- 2. Does the delivery method (OJ vs VC) significantly impact tooth growth?

> Is orange juice more effective than pure ascorbic acid?

- 3. Does the dose level (0.5, 1, 2 mg/day) significantly affect tooth growth?

> Is there a dose-response relationship?

## Interaction Questions:
- 4. Is there an interaction between delivery method and dose?

> Does the effect of delivery method depend on the dose level?

> Does the dose-response relationship differ between OJ and VC?

## Assumption Checking Questions:
- 5. Is the data normally distributed within each group?

> Shapiro-Wilk tests

> QQ plots

- 6. Is homogeneity of variance satisfied?

> Levene's test

> Bartlett's test

- 7. Are there any outliers influencing the results?

- 8. Is the sample size adequate for the tests being used?

## Statistical Significance Questions:
- 9. What is the p-value for each main effect and interaction?

- 10. Which group differences are statistically significant (α=0.05)?

- 11. Are the results robust after multiple comparison corrections?

## 1. Libraries

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Alternatively, you can create the dataset manually:
# data = {
#     'len': [4.2, 11.5, 7.3, 5.8, 6.4, 10.0, 11.2, 11.2, 5.2, 7.0, 16.5, 16.5, 15.2, 17.3, 22.5, 17.3, 13.6, 14.5, 18.8, 15.5, 23.6, 18.5, 33.9, 25.5, 26.4, 32.5, 26.7, 21.5, 23.3, 29.5, 15.2, 21.5, 17.6, 9.7, 14.5, 10.0, 8.2, 9.4, 16.5, 9.7, 19.7, 23.3, 23.6, 26.4, 20.0, 25.2, 25.8, 21.2, 14.5, 27.3, 25.5, 26.4, 22.4, 24.5, 24.8, 30.9, 26.4, 27.3, 29.4, 23.0],
#     'supp': ['VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'VC', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ', 'OJ'],
#     'dose': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]
# }
# df = pd.DataFrame(data)

print("Dataset Overview:")
print(df.head())
print(f"\nDataset shape: {df.shape}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nBasic info:")
print(df.info())

## 2. Dataset

## 3. Descriptive Statistic

In [ ]:
print("=== DESCRIPTIVE STATISTICS ===")

# Overall statistics
print("\n1. OVERALL TOOTH LENGTH STATISTICS:")
overall_stats = df['len'].describe()
print(overall_stats)

# Statistics by supplement type
print("\n2. STATISTICS BY SUPPLEMENT TYPE:")
supplement_stats = df.groupby('supp')['len'].agg(['count', 'mean', 'std', 'min', 'max', 'median'])
print(supplement_stats)

# Statistics by dose level
print("\n3. STATISTICS BY DOSE LEVEL:")
dose_stats = df.groupby('dose')['len'].agg(['count', 'mean', 'std', 'min', 'max', 'median'])
print(dose_stats)

# Combined statistics by supplement and dose
print("\n4. COMBINED STATISTICS BY SUPPLEMENT AND DOSE:")
combined_stats = df.groupby(['supp', 'dose'])['len'].agg(['count', 'mean', 'std', 'min', 'max', 'median'])
print(combined_stats)

# Calculate confidence intervals
print("\n5. 95% CONFIDENCE INTERVALS:")
def confidence_interval(data, confidence=0.95):
    n = len(data)
    mean = np.mean(data)
    std_err = stats.sem(data)
    h = std_err * stats.t.ppf((1 + confidence) / 2, n - 1)
    return mean - h, mean + h

print("Confidence intervals for each group:")
for supp in ['OJ', 'VC']:
    for dose in [0.5, 1.0, 2.0]:
        group_data = df[(df['supp'] == supp) & (df['dose'] == dose)]['len']
        ci_lower, ci_upper = confidence_interval(group_data)
        print(f"{supp} - {dose}mg: ({ci_lower:.2f}, {ci_upper:.2f})")

## 4. Data Visualization

In [ ]:
print("=== DATA VISUALIZATION ===")

# Create a comprehensive visualization dashboard
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Boxplot by supplement and dose
sns.boxplot(data=df, x='dose', y='len', hue='supp', ax=axes[0,0])
axes[0,0].set_title('A. Tooth Length Distribution by Dose and Supplement')
axes[0,0].set_xlabel('Dose (mg/day)')
axes[0,0].set_ylabel('Tooth Length (mm)')

# 2. Violin plot for distribution details
sns.violinplot(data=df, x='dose', y='len', hue='supp', split=True, ax=axes[0,1])
axes[0,1].set_title('B. Detailed Distribution by Dose and Supplement')
axes[0,1].set_xlabel('Dose (mg/day)')
axes[0,1].set_ylabel('Tooth Length (mm)')

# 3. Bar plot with confidence intervals
sns.barplot(data=df, x='dose', y='len', hue='supp', ax=axes[0,2], ci=95)
axes[0,2].set_title('C. Mean Tooth Length with 95% Confidence Intervals')
axes[0,2].set_xlabel('Dose (mg/day)')
axes[0,2].set_ylabel('Mean Tooth Length (mm)')

# 4. Interaction plot
dose_supp_means = df.groupby(['dose', 'supp'])['len'].mean().unstack()
dose_supp_means.plot(ax=axes[1,0], marker='o', linewidth=2.5)
axes[1,0].set_title('D. Interaction: Mean Tooth Length by Dose and Supplement')
axes[1,0].set_xlabel('Dose (mg/day)')
axes[1,0].set_ylabel('Mean Tooth Length (mm)')
axes[1,0].legend(title='Supplement')
axes[1,0].grid(True, alpha=0.3)

# 5. Histogram of tooth lengths
df['len'].hist(bins=15, ax=axes[1,1], alpha=0.7, edgecolor='black')
axes[1,1].set_title('E. Distribution of All Tooth Lengths')
axes[1,1].set_xlabel('Tooth Length (mm)')
axes[1,1].set_ylabel('Frequency')

# 6. Scatter plot with regression lines
sns.regplot(data=df[df['supp']=='OJ'], x='dose', y='len', 
            ax=axes[1,2], label='OJ', scatter_kws={'alpha':0.6})
sns.regplot(data=df[df['supp']=='VC'], x='dose', y='len', 
            ax=axes[1,2], label='VC', scatter_kws={'alpha':0.6})
axes[1,2].set_title('F. Dose-Response Relationship by Supplement')
axes[1,2].set_xlabel('Dose (mg/day)')
axes[1,2].set_ylabel('Tooth Length (mm)')
axes[1,2].legend()

plt.tight_layout()
plt.show()

# Additional detailed plot
plt.figure(figsize=(10, 6))
sns.pointplot(data=df, x='dose', y='len', hue='supp', 
              ci=95, dodge=0.2, capsize=0.1, errwidth=1.5)
plt.title('Mean Tooth Length with 95% Confidence Intervals\n(Detailed View)')
plt.xlabel('Dose (mg/day)')
plt.ylabel('Tooth Length (mm)')
plt.legend(title='Supplement Type')
plt.grid(True, alpha=0.3)
plt.show()

## Assumption Checking For Statistical Tests

In [ ]:
print("=== ASSUMPTION CHECKING ===")

# 1. Normality tests using Shapiro-Wilk
print("\n1. NORMALITY TESTS (Shapiro-Wilk):")
print("H₀: Data is normally distributed")
print("H₁: Data is not normally distributed")
print("α = 0.05")

for supp in ['OJ', 'VC']:
    for dose in [0.5, 1.0, 2.0]:
        group_data = df[(df['supp'] == supp) & (df['dose'] == dose)]['len']
        stat, p_value = stats.shapiro(group_data)
        normality = "Normal" if p_value > 0.05 else "Not Normal"
        print(f"{supp} - {dose}mg: W={stat:.4f}, p={p_value:.4f} ({normality})")

# 2. Homogeneity of variance tests
print("\n2. HOMOGENEITY OF VARIANCE TESTS:")

# Levene's test between supplements
levene_supp_stat, levene_supp_p = stats.levene(
    df[df['supp'] == 'OJ']['len'],
    df[df['supp'] == 'VC']['len']
)
print(f"Levene's test (between supplements): F={levene_supp_stat:.4f}, p={levene_supp_p:.4f}")

# Levene's test between doses
dose_groups = [df[df['dose'] == dose]['len'] for dose in [0.5, 1.0, 2.0]]
levene_dose_stat, levene_dose_p = stats.levene(*dose_groups)
print(f"Levene's test (between doses): F={levene_dose_stat:.4f}, p={levene_dose_p:.4f}")

# 3. QQ plots for normality check
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Q-Q Plots for Normality Assessment', fontsize=16)

plot_idx = 0
for supp in ['OJ', 'VC']:
    for dose in [0.5, 1.0, 2.0]:
        group_data = df[(df['supp'] == supp) & (df['dose'] == dose)]['len']
        stats.probplot(group_data, dist="norm", plot=axes[plot_idx//3, plot_idx%3])
        axes[plot_idx//3, plot_idx%3].set_title(f'{supp} - {dose}mg')
        plot_idx += 1

plt.tight_layout()
plt.show()

## 5. Inferential Statistic

In [ ]:
print("=== INFERENTIAL STATISTICS ===")

# 1. T-tests for supplement comparison
print("\n1. T-TESTS FOR SUPPLEMENT COMPARISON:")

# Independent t-test between all OJ vs VC
oj_data = df[df['supp'] == 'OJ']['len']
vc_data = df[df['supp'] == 'VC']['len']

t_stat, p_value = stats.ttest_ind(oj_data, vc_data)
print(f"Overall OJ vs VC: t({len(oj_data)+len(vc_data)-2}) = {t_stat:.4f}, p = {p_value:.4f}")

# T-tests at each dose level
print("\nT-tests at each dose level:")
for dose in [0.5, 1.0, 2.0]:
    oj_dose = df[(df['supp'] == 'OJ') & (df['dose'] == dose)]['len']
    vc_dose = df[(df['supp'] == 'VC') & (df['dose'] == dose)]['len']
    t_stat, p_value = stats.ttest_ind(oj_dose, vc_dose)
    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "ns"
    print(f"Dose {dose}mg: t({len(oj_dose)+len(vc_dose)-2}) = {t_stat:.4f}, p = {p_value:.4f} {significance}")

# 2. One-way ANOVA for dose effect
print("\n2. ONE-WAY ANOVA FOR DOSE EFFECT:")
f_stat, p_value = stats.f_oneway(
    df[df['dose'] == 0.5]['len'],
    df[df['dose'] == 1.0]['len'],
    df[df['dose'] == 2.0]['len']
)
print(f"One-way ANOVA (dose): F(2,57) = {f_stat:.4f}, p = {p_value:.4f}")

# 3. Two-way ANOVA
print("\n3. TWO-WAY ANOVA:")
model = ols('len ~ C(supp) + C(dose) + C(supp):C(dose)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("Two-way ANOVA table:")
print(anova_table)